In [15]:
import requests
import json

/home/enbrands/miniconda3/envs/xiao/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.14) or chardet (5.2.0)/charset_normalizer (2.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


# 测试话术存储模块

In [18]:
url = "http://121.201.110.83:30504/sales_template_save"
post_data = {
    "industry_id": "默认",
    "template_id": "默认",
    "template_content": "默认销售"
}
json_post_data = json.dumps(post_data)
response = requests.post(url, data=json_post_data)


In [19]:
response.json()

{'status': 200, 'msg': 'success'}

# 测试模型回复模块

In [21]:
url = "http://121.201.110.83:30504/model_reply"
post_data = {
    "industry_id": "医美",
    "template_id": "project_1",
    "chat_id": "test_1",
    "user_input": "想问问皮肤相关的有啥项目"
}
json_post_data = json.dumps(post_data)
response = requests.post(url, data=json_post_data)


In [22]:
response.json()

{'status': 200,
 'msg': 'Model reply success',
 'model_reply': '{"customer_intention": "客户想了解皮肤相关的医美项目", "reply": "您好！很高兴您对皮肤相关的项目感兴趣。我们提供多种皮肤护理和改善的方案，比如水光针、激光美容、化学换肤、微针美容等。每个项目都有其独特的效果和适应人群，您可以告诉我您的具体需求，比如想改善的皮肤问题，我会为您推荐最合适的项目。期待您的回复！"}',
 'input_tokens': 256,
 'output_tokens': 111}

In [11]:
import json
a = '{"customer_intention": "", "reply": "ds"}'
c = json.loads(a)

In [14]:
c['reply']

'ds'

# 测试对话效果

## 获取对话记录

In [25]:
from utils.connect_mongo import _init_mongo_connect
import json

In [36]:
db_name = 'smart_salesman'
db_test = _init_mongo_connect(db_name)
dialogue_test = db_test['user_dialogue_db']

In [37]:
chat_ids = dialogue_test.distinct('chat_id')

In [40]:
chat_ids

['6pdPwSrH',
 '7HuGI87G',
 '882rJh9j',
 '8puWPCEh',
 '9my1223u',
 'A4nQ86vt',
 'Cu9Dtydf',
 'G67T1gsq',
 'GsG8q8Bp',
 'Je7osb32',
 'K2iWWAOM',
 'M5DfcyeW',
 'Mb2Tn81V',
 'QJ0HBMpG',
 'ZIZf5OHx',
 'bU4IHeow',
 'ffsi4Wdc',
 'kM9FzbM7',
 'q3n4954a',
 'r1k72N5X',
 'test_1',
 'uUjAxQMZ',
 'wzOZiNDt',
 'zWm9obmr',
 'zi2SuY6Y']

In [15]:
list(dialogue_test.find_one().keys())

['_id', 'chat_id', 'user_input', 'model_reply', 'insert_time']

In [45]:
target_chat_id = 'dFxp6zoU'
dialogue_record = dialogue_test.find({'chat_id': target_chat_id})
dialogue_res = []
for dialogue in dialogue_record:
    user_input = dialogue.get('user_input', 'no_input')
    # model_reply_with_intention = dialogue.get('model_reply', "no_reply")
    model_reply = json.loads(dialogue.get('model_reply', 'no_reply'))['reply']
    dialogue_res.append([{"role": "user", "content": user_input}, {"role": "assistant", "content": model_reply}])

In [46]:
dialogue_res

[[{'role': 'user', 'content': '你好'},
  {'role': 'assistant',
   'content': '你好！很高兴收到你的消息。请问你对我们的产品有什么具体的兴趣或者需求呢？我很乐意为你提供详细的信息！如果方便的话，可以告诉我你的名字和联系方式吗？这样我可以更好地帮助你。'}],
 [{'role': 'user', 'content': '有什么瘦脸的项目吗'},
  {'role': 'assistant',
   'content': '当然有的！我们提供多种瘦脸项目，包括注射瘦脸针、面部塑形和美容仪器等，能够帮助你实现理想的脸型。每个项目都有其独特的效果和适应人群，您对哪种方式比较感兴趣呢？如果方便的话，可以告诉我您的名字和联系方式吗？这样我可以为您提供更详细的信息和个性化的建议！'}],
 [{'role': 'user', 'content': '没啥特别感兴趣的，我先留个电话，回头来店里吧'},
  {'role': 'assistant',
   'content': '非常感谢你愿意留下电话！我理解有时候需要时间来考虑。为了更好地为你提供服务，能否请你告诉我你的名字呢？这样我可以在你来店里时为你准备一些个性化的建议和信息。如果你有任何问题或者需要了解的项目，随时都可以联系我哦！期待你的光临！'}],
 [{'role': 'user', 'content': '姓李，电话是13928193842'},
  {'role': 'assistant',
   'content': '谢谢李先生/女士！很高兴您愿意留下联系方式。为了更好地为您提供服务，您是否有特别关注的瘦脸项目或者其他美容需求呢？这样我可以为您准备一些个性化的建议和信息，期待您的光临！如果您有任何问题，随时可以联系我哦！'}]]